In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
data = pd.read_csv('drive/My Drive/deeplearningicp5/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [0]:
def createmodel():
    model = Sequential()
    lstm_out=196
    embed_dim=128
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary())

In [11]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [12]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 19s - loss: 0.8329 - accuracy: 0.6466
Epoch 2/20
 - 18s - loss: 0.6844 - accuracy: 0.7034
Epoch 3/20
 - 18s - loss: 0.6164 - accuracy: 0.7343
Epoch 4/20
 - 17s - loss: 0.5807 - accuracy: 0.7562
Epoch 5/20
 - 18s - loss: 0.5373 - accuracy: 0.7798
Epoch 6/20
 - 18s - loss: 0.5033 - accuracy: 0.7933
Epoch 7/20
 - 18s - loss: 0.4685 - accuracy: 0.8057
Epoch 8/20
 - 18s - loss: 0.4298 - accuracy: 0.8237
Epoch 9/20
 - 18s - loss: 0.4005 - accuracy: 0.8351
Epoch 10/20
 - 18s - loss: 0.3784 - accuracy: 0.8442
Epoch 11/20
 - 18s - loss: 0.3524 - accuracy: 0.8563
Epoch 12/20
 - 18s - loss: 0.3306 - accuracy: 0.8654
Epoch 13/20
 - 17s - loss: 0.3178 - accuracy: 0.8700
Epoch 14/20
 - 17s - loss: 0.2968 - accuracy: 0.8782
Epoch 15/20
 - 18s - loss: 0.2860 - accuracy: 0.8839
Epoch 16/20
 - 18s - loss: 0.2682 - accuracy: 0.8898
Epoch 17/20
 - 18s - loss: 0.2656 - accuracy: 0.8899
Epoch 18/20
 - 17s - loss: 0.2570 - accuracy: 0.8966
Epoch 19/20
 - 18s - loss: 0.2527 - accuracy: 0.8971
Ep

In [0]:
model.save('jaswanth.h5')

In [14]:
from keras.models import load_model
model= load_model('jaswanth.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [15]:
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

2
Positive


2. Apply GridSearchCV on the source code provided in the class

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 44s - loss: 0.8326 - accuracy: 0.6464
Epoch 1/1
 - 45s - loss: 0.8264 - accuracy: 0.6439
Epoch 1/1
 - 45s - loss: 0.8376 - accuracy: 0.6384
Epoch 1/1
 - 46s - loss: 0.8309 - accuracy: 0.6421
Epoch 1/1
 - 47s - loss: 0.8274 - accuracy: 0.6445
Epoch 1/2
 - 45s - loss: 0.8329 - accuracy: 0.6412
Epoch 2/2
 - 46s - loss: 0.6907 - accuracy: 0.7112
Epoch 1/2
 - 45s - loss: 0.8278 - accuracy: 0.6433
Epoch 2/2
 - 46s - loss: 0.6895 - accuracy: 0.7108
Epoch 1/2
 - 47s - loss: 0.8306 - accuracy: 0.6430
Epoch 2/2
 - 47s - loss: 0.6834 - accuracy: 0.7158
Epoch 1/2
 - 47s - loss: 0.8310 - accuracy: 0.6416
Epoch 2/2
 - 46s - loss: 0.6792 - accuracy: 0.7137
Epoch 1/2
 - 45s - loss: 0.8252 - accuracy: 0.6468
Epoch 2/2
 - 46s - loss: 0.6809 - accuracy: 0.7077
Epoch 1/1
 - 23s - loss: 0.8395 - accuracy: 0.6396
Epoch 1/1
 - 24s - loss: 0.8437 - accuracy: 0.6394
Epoch 1/1
 - 24s - loss: 0.8341 - accuracy: 0.6404
Epoch 1/1
 - 24s - loss: 0.8440 - accuracy: 0.6399
Epoch 1/1
 - 24s - loss: 0.8308

3. Apply the code on spamdata set availablein thesourcecode (text classification on the spam.csvdata set)

In [0]:
data1 = pd.read_csv('drive/My Drive/deeplearningicp5/spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]

In [0]:
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='sigmoid'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [24]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 37s - loss: 0.2359 - accuracy: 0.9218
Epoch 2/5
 - 35s - loss: 0.0492 - accuracy: 0.9861
Epoch 3/5
 - 36s - loss: 0.0271 - accuracy: 0.9912
Epoch 4/5
 - 35s - loss: 0.0161 - accuracy: 0.9957
Epoch 5/5
 - 36s - loss: 0.0105 - accuracy: 0.9971
0.09783113029092406
0.9825992584228516
['loss', 'accuracy']
